In [1]:
import torch
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2

from maderapp.model import TimberEfficientNet, TimberEfficientNetNS, TimberMobileNet, TimberResNet
from maderapp.trainer import trainer

In [6]:
metadata = pd.read_csv("../metadata.csv", header=None)
metadata = pd.DataFrame({
    "x1": ["Almendro/Almendro (1).jpg", "Almendro/Almendro (9).jpg"] * 4,
    "x2": [0,0] * 4 
})
train_trans = A.Compose(
        [
            A.Resize(224, 224),
            A.RandomCrop(width=224, height=224),
            A.augmentations.geometric.rotate.Rotate(),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ]
    )

val_trans = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2(),
    ]
)

"""
for model_class, model_name in zip(
    [TimberEfficientNet, TimberEfficientNetNS, TimberMobileNet],
    ["efficientNet_15", "efficientNet-NS_15", "MobileNet_15"],
):
    model = model_class(num_classes=out_features)
    trainer(metadata, model, None, train_trans, val_trans, model_name)
"""

BATCH_SIZE = 4
OUT_FEATURES = 1
NUM_EPOCHS = 2
REQUIRE_GRAD = True
VALIDATION = True
MAX_EPOCHS = 15
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

for kfold in range(4):
    # Load model
    model = TimberMobileNet(num_classes=OUT_FEATURES)
    trainer(metadata,"../data", model, kfold, train_trans, val_trans, "MobileNet", BATCH_SIZE, NUM_EPOCHS, VALIDATION, DEVICE, "model_logs")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type         | Params
-----------------------------------------------
0 | train_acc     | Accuracy     | 0     
1 | test_acc      | Accuracy     | 0     
2 | train_f1score | F1Score      | 0     
3 | test_f1score  | F1Score      | 0     
4 | softmax       | Softmax      | 0     
5 | model         | EfficientNet | 2.2 M 
-----------------------------------------------
1.3 K     Trainable params
2.2 M     Non-trainable params
2.2 M     Total params
8.901     Total estimated model params size (MB)


Number no trainable parameters: 2223872 
Number trainable parameters: 1281
training fold=0
Epoch 1: 100%|██████████| 4/4 [00:02<00:00,  1.40it/s, loss=0, v_num=2, train_acc=0.000, train_f1score=0.000, train_loss=0.000, val_acc=0.000, val_f1score=0.000, val_loss=0.000]


ZeroDivisionError: float division by zero